In [73]:
#Import these for Python2 & Python 3 support
from __future__ import absolute_import
from __future__ import print_function

# set some ipython notebook properties
%matplotlib inline

# set degree of verbosity (adapt to INFO for more verbose output)
import logging
logging.basicConfig(level=logging.WARNING)

# set figure sizes
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)
#pylab.plot([1,2,3],[4,5,6])

# set display width for pandas data frames
from scipy.io import mmread
import numpy as np
import pandas as pd

pd.set_option('display.width', 1000)

from pyannotables import tables
import pegasus as pg

from pysnptools.snpreader import Dat, Bed, Pheno, SnpData
import pysnptools.util as pstutil

# import the algorithm
from fastlmm.association import single_snp

## PySNPTools Toy Data

In [2]:
pysnp_path = 'software/PySnpTools/pysnptools/examples/'

In [ ]:
snpdata = Bed(pysnp_path + 'toydata.bed',count_A1=False)[:,:10].read()  # Read first 10 snps from Bed format
pstutil.create_directory_if_necessary("temp/toydata10.dat")
Dat.write("temp/toydata10.dat", snpdata) 

In [ ]:
# define file names
snp_reader_bed = Bed(pysnp_path + 'toydata.bed',count_A1=False)

In [ ]:
# define file names
snp_reader_dat = Dat("temp/toydata10")

In [ ]:
snp_reader_bed.shape, snp_reader_dat.shape

## 10X PBMC Data 

http://zzz.bwh.harvard.edu/plink/data.shtml
  - .dat: genes by cells (first 3 columns are id and meta data) sid, paternal, maternal?
  - .fam: cell data and metadata    fid, iid, paternal, maternal, sex, phenotype?
  - .map: gene/genomic map of genes    chr, geneid, cM(leave as 0), bp (bp start of gene) 

In [3]:
pbmc_path = 'data/pbmc3k/filtered_gene_bc_matrices/hg19/'

In [4]:
adata = pg.read_input(pbmc_path + 'matrix.mtx')

2020-04-15 16:09:57,292 - pegasus - INFO - Detected mtx file in 10x v2 format.


INFO:pegasus:Detected mtx file in 10x v2 format.


2020-04-15 16:09:57,453 - pegasus - INFO - Time spent on 'read_input' = 4.80s.


INFO:pegasus:Time spent on 'read_input' = 4.80s.


In [5]:
pg.qc_metrics(adata)
pg.filter_data(adata)

2020-04-15 16:24:22,450 - pegasus - INFO - After filtration, 2481/2700 cells and 16543/32738 genes are kept. Among 16543 genes, 14616 genes are robust.


INFO:pegasus:After filtration, 2481/2700 cells and 16543/32738 genes are kept. Among 16543 genes, 14616 genes are robust.


In [6]:
pg.log_norm(adata)
pg.highly_variable_features(adata, consider_batch = False)
pg.pca(adata)

2020-04-15 16:24:24,566 - pegasus - INFO - Time spent on 'log_norm' = 0.06s.


INFO:pegasus:Time spent on 'log_norm' = 0.06s.


2020-04-15 16:24:24,632 - pegasus - INFO - 2000 highly variable features have been selected.


INFO:pegasus:2000 highly variable features have been selected.


2020-04-15 16:24:24,634 - pegasus - INFO - Time spent on 'highly_variable_features' = 0.07s.


INFO:pegasus:Time spent on 'highly_variable_features' = 0.07s.


2020-04-15 16:24:24,938 - pegasus - INFO - PCA is done. Time spent = 0.29s.


INFO:pegasus:PCA is done. Time spent = 0.29s.


In [18]:
adata_filt = adata[:,adata.var.hvf_rank.between(0, 100)]

In [19]:
adata_filt

View of AnnData object with n_obs × n_vars = 2481 × 40 
    obs: 'Channel', 'passed_qc', 'n_genes', 'n_counts', 'percent_mito'
    var: 'gene_ids', 'n_cells', 'percent_cells', 'robust', 'highly_variable_features', 'mean', 'var', 'hvf_loess', 'hvf_rank'
    uns: 'genome', 'Channels', 'fmat_highly_variable_features', 'PCs', 'pca'
    obsm: 'X_pca'

In [20]:
## genes = pd.read_csv(pbmc_path + 'genes.tsv', sep = '\t', names = ['id', 'symbol'])
genes = adata_filt.var

In [29]:
## barcodes = pd.read_csv(pbmc_path + 'barcodes.tsv', sep = '\t', names = ['barcode'])
adata_filt_obs = adata_filt[adata_filt.obs['n_genes'].between(1000,1100)]
barcodes = adata_filt_obs.obs

In [31]:
adata_filt_obs.X

<166x40 sparse matrix of type '<class 'numpy.float32'>'
	with 4055 stored elements in Compressed Sparse Row format>

In [34]:
## mat = mmread(pbmc_path + 'matrix.mtx')
## mat_t = mat.todense()[:100, :10].T
mat = adata_filt_obs.X.todense()

In [51]:
mat.shape, barcodes.shape, genes.shape

((166, 40), (166, 5), (40, 9))

In [39]:
## iid = [['fam0','iid0'],['fam0','iid1']]  ## use the fam, fid, to indicate subject and iid as cell barcode
iid_bc = [['fam0', b] for b in barcodes.index.tolist()]

In [42]:
sid_genes = genes['gene_ids'].tolist()

In [43]:
sid_genes[:5]

['ENSG00000163220',
 'ENSG00000143546',
 'ENSG00000197956',
 'ENSG00000196154',
 'ENSG00000158869']

In [44]:
ensembl = tables['homo_sapiens-GRCh37-ensembl99']

In [45]:
pos_genes = ensembl[ensembl.index.isin(sid_genes)][['Chromosome', 'Start', 'Start']].values.tolist()

In [50]:
## using transcriptome to calculate the kinship matrices
## snpdata = SnpData(iid=[['fam0','iid0'],['fam0','iid1']], sid=['snp334','snp349','snp921'], val=[[0.,2.,0.],[0.,1.,2.]])
## pos - [chr, start, end]
snpdata = SnpData(iid = iid_bc, sid = sid_genes, val = mat, pos = pos_genes)

In [ ]:
Dat.write("temp/pbmc100.dat", snpdata)

In [66]:
## predicting genes / gene exptession
phenodata = SnpData(iid = iid_bc, sid = [sid_genes[9]], val = mat[:,9])

In [67]:
mat[:,9]

matrix([[5.6958785],
        [6.5767665],
        [6.3086066],
        [5.055046 ],
        [4.6966333],
        [6.088486 ],
        [5.7743163],
        [5.5693636],
        [6.388266 ],
        [6.919407 ],
        [5.523676 ],
        [5.36171  ],
        [5.9151874],
        [5.641898 ],
        [6.4863033],
        [4.5938997],
        [5.8909545],
        [6.781117 ],
        [5.3116317],
        [3.6031659],
        [6.7126675],
        [6.7188444],
        [5.3641763],
        [5.4205704],
        [5.5249   ],
        [4.7599554],
        [6.505121 ],
        [6.190226 ],
        [6.946568 ],
        [5.73938  ],
        [4.9049673],
        [5.8472304],
        [4.2896705],
        [6.988319 ],
        [6.382187 ],
        [5.763498 ],
        [6.8991337],
        [6.4160514],
        [3.3722913],
        [6.609569 ],
        [5.887284 ],
        [6.4990454],
        [6.113965 ],
        [3.745718 ],
        [6.645214 ],
        [5.5924025],
        [6.7733335],
        [6.35

In [ ]:
## results_df = single_snp(test_snps, pheno_fn, K0=G0, K1=G1, covar=cov_fn, GB_goal=2, count_A1=True)
results_df = single_snp(test_snps = Dat(snpdata), pheno = Pheno(phenodata), K0 = Dat(snpdata))